In [1]:
# Install necessary libraries
!pip install unidecode --quiet
!pip install openai --quiet
!pip install python-dotenv --quiet
!pip install pandas --quiet
!pip install json --quiet
!pip install requests --quiet
!pip install groq --quiet
!pip install jsonref --quiet
!pip install -U instructor --quiet


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.5/235.5 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 336.7/336.7 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 1.4 MB/s eta 0:00:00
ERROR: Could not find a version that satisfies the requirement json (from versions: none)
ERROR: No matching distribution found for json
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.5/103.5 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 327.6/327.6 kB 11.7 MB/s eta 0:00:00


In [5]:
# Import required libraries
from unidecode import unidecode
import pandas as pd
import json
import openai
from pydantic import (
    BaseModel,
    BeforeValidator,
    PlainSerializer,
    InstanceOf,
    WithJsonSchema,
    Field
)
from groq import Groq
import instructor
# Load environment variables
from google.colab import userdata
from io import StringIO
from typing import Annotated, Any, List, Optional

In [3]:
# Read the CSV file into a DataFrame
df = pd.read_csv("/PGP_sourced_data/documents.csv",on_bad_lines='skip')
#df = pd.read_csv("/content/documents.csv",on_bad_lines='skip') #IF DOING FROM COLAB USE THIS

df = df[['pgpid','type','description']]
print(df.head())

   pgpid            type                                        description
0    444          Letter  Fragment of a draft letter in the name of the ...
1    445  Legal document  Fragment of a legal document in which the sign...
2    446  Legal document  A writ of qiddushin (betrothal), Tyre, ca. 101...
3    447  Legal document  Legal document. According to Ashtor, "an agree...
4    448          Letter  Letter from Mufaḍḍal, probably in Fustat, to A...


In [ ]:
# Filter rows based on specific substrings
df['count'] = (df['description'].str.count(' b. ') +
               df['description'].str.count(' ibn ') +
               df['description'].str.count(' the judge ') +
               df['description'].str.count(' al') +
               df['description'].str.count(' abu'))
df = df[df['count'] >= 2]

In [12]:
class LegalWithWitnesses(BaseModel):
    pgpid: int
    original_location:  Optional[str] = Field(..., description="The location where the document was composed")
    validation_location: Optional[str] = Field(..., description="The location where the document was validated, if mentioned")
    date: str = Field(..., description="The date the document was authored as stated in the text")
    date_standard: str = Field(..., description="The date of the letter in the format {YYYY-MM-DD}")
    author: str = Field(..., description="The name of the author who composed the document")
    scribe: Optional[str] =Field(..., description = "The name of the scribe who composed the document, if seperate from the author")
    original_witnesses: Optional[List[str]] = Field(..., description="The names of the witnesses who signed the document")
    verifying_witnesses: Optional[List[str]] = Field(..., description="The names of the witnesses who validation the document, if there is a seperate validation")
    names: List[str] = Field(..., description="A list of names in the document")
    note: Optional[str] = Field(..., description="Any notes or comments about the extracted information")

class Letter(BaseModel):
    pgpid: int
    names: List[str] = Field(..., description="A list of names in the document")
    start_location: str = Field(..., description="The location where the document was composed")
    end_location: str = Field(..., description="The location where the document sent")
    mentioned_locations: List[str] = Field(..., description="A list of locations mentioned in the document, if seperate from the start and end destinations")
    date: str = Field(..., description="The date the document was authored as stated in the text. a general time period should be {startdate}/{enddate}")
    date_standard: str = Field(..., description="The date of the letter in the format {YYYY-MM-DD}, a general time period should be {startdate}/{enddate}")
    author: str =Field(..., description="The name of the author who composed the document")
    author_type: str =Field(...,description="type of author: identifiable_individual, unidentifiable_individual, community, other")
    scribe: Optional[str] =Field(..., description = "The name of the scribe who composed the document, if seperate from the author")
    recipient: str = Field(..., description="The name of the recipient who received the document")
    recipient_type: str = Field(...,description="type of recipient: identifiable_individual, unidentifiable_individual, community, other")
    note: str = Field(..., description = "Note anything that you are uncertain of or want to be mentioned")


In [ ]:
new_df = df[df['type'] == 'Legal document']
#new_df = df[df['type'] == 'Letter']
#IF DOING


In [ ]:

client = Groq(
    api_key=userdata.get('GROQ_API_KEY'),
)

client = instructor.from_groq(client, mode=instructor.Mode.TOOLS)
def get_info(prompt):

  resp = client.chat.completions.create(
      model="llama3-70b-8192",
      messages=[
          {
              "role": "user",
              "content": prompt,
          }
      ],
      response_model=LegalWithWitnesses, #IF DOING LEGAL, KEEP AS LegalWithWitnesses, IF DOING LETTERS CHANGE TO Letter
  )
  return json.loads(resp.model_dump_json(indent=2))

js = []
for doc in new_df.to_dict('records'):
  geniza_test = doc['description']
  pgp_id = doc['pgpid']
  prompt = f"""
    You are a Cairo Geniza expert tasked with extracting historical data from the provided legal text description.
    Please extract and return the following details in JSON format:



     Guidelines:
    1. Extract all relevant information based on the Pydantic model.
    2. Use 'N/A' for any unknown or unavailable information.
    3. Ensure all names include patronymics and descriptors when available.
    4. For date ranges, use the format 'startdate/enddate' in both 'date' and 'date_standard' fields.
    5. Include all mentioned locations in 'mentioned_locations', except those already listed in start_location and end_location.
    6. Names typically include patronymics such as 'b.', 'ibn', or descriptors like 'al-[location]'. Be sure to include full names instead of partial names when they are available. Do not list names twice if they are a full name and partial name (e.g. do not list Barhun b. Ishaq and Barhun if they are clearly the same person)

    Input: pgpid: '{pgp_id}', description: '{geniza_test}'
    Output:
    """
  data = get_info(prompt)
  print(data)
  js.append(data)




{'pgpid': 445, 'original_location': 'N/A', 'validation_location': 'N/A', 'date': 'N/A', 'date_standard': 'N/A', 'author': 'N/A', 'scribe': 'N/A', 'original_witnesses': ['Toviyya b. Daniel'], 'verifying_witnesses': ['Toviyya b. Daniel'], 'names': ['Toviyya b. Daniel'], 'note': 'Fragment of a legal document'}
{'pgpid': 446, 'original_location': 'Tyre', 'validation_location': 'N/A', 'date': '1011-1037', 'date_standard': '1011-1037', 'author': 'N/A', 'scribe': 'N/A', 'original_witnesses': [], 'verifying_witnesses': [], 'names': [], 'note': 'N/A'}
{'pgpid': 447, 'original_location': 'N/A', 'validation_location': 'N/A', 'date': 'second half of the 15th century/second half of the 15th century', 'date_standard': '1450-01-01/1499-12-31', 'author': 'N/A', 'scribe': 'N/A', 'original_witnesses': ['Farajallāh b. Yaʿqūb'], 'verifying_witnesses': ['Muhadhdhab b. Raḍiyy', 'Shuqra', 'Farajallāh b. Yaʿqūb'], 'names': ['Muhadhdhab b. Raḍiyy', 'Shuqra', 'Farajallāh b. Yaʿqūb'], 'note': 'N/A'}
{'pgpid': 44

In [10]:
js2 = pd.DataFrame(js)
#should remember to save the documents and rewrite this
js2.to_json('all_legal_20240807.json', orient='records')